In [8]:
class_id_to_name = {
    0:  ('road', [28, 42, 168]),
    1:  ('pool', [0, 50, 89]),
    2:  ('vegetation', [107, 142, 35]),
    3:  ('roof', [70, 70, 70]),
    4:  ('wall', [102, 102, 156]),
    5:  ('window', [254, 228, 12]),
    6:  ('person', [255, 22, 96]),
    7:  ('dog', [102, 51, 0]),
    8:  ('car', [9, 143, 150]),
    9:  ('bicycle', [119, 11, 32]),
    10: ('tree', [51, 51, 0]),
    11: ('truck', [160, 160, 60]),   # added truck
    12: ('bus', [200, 80, 80]),      # added bus
    13: ('vehicle', [20, 80, 80]),      # added bus
}

In [9]:
import cv2
from tqdm.auto import tqdm
from ultralytics import YOLO
import os
from pathlib import Path

In [10]:


def process_frame_1(frame, yolo_model, w, h, class_id_to_name, conf_threshold=0.5):
    annotated = frame.copy()
    results = yolo_model(annotated, conf=conf_threshold, verbose=False)[0]

    boxes = results.boxes.xyxy.cpu().numpy()
    class_ids = results.boxes.cls.cpu().numpy()
    confidences = results.boxes.conf.cpu().numpy()

    for box, cls_id, confidence in zip(boxes, class_ids, confidences):
        x1, y1, x2, y2 = map(int, box)
        class_name, color = class_id_to_name[int(cls_id)]
        cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
        cv2.putText(annotated, f"{class_name} {confidence:.2f}", (x1, max(y1 - 10, 10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

    return annotated, boxes, class_ids



# ========== VIDEO CAPTURE ==========
def setup_video_capture_1(video_path):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return cap, total_frames, fps, w, h

# ========== MAIN FUNCTION ==========
def videos_predictions(yolo_weights_path, class_id_to_name, video_dir='videos', output_base='./datatsets/opt', max_frames=None):

    print("Model Used ::", yolo_weights_path)

    yolo_model = YOLO(yolo_weights_path)

    image_out_dir = os.path.join(output_base, 'images')
    label_out_dir = os.path.join(output_base, 'labels')
    output_video_dir = os.path.join(output_base, 'output')

    os.makedirs(image_out_dir, exist_ok=True)
    os.makedirs(label_out_dir, exist_ok=True)
    os.makedirs(output_video_dir, exist_ok=True)

    for video_file in tqdm(sorted(os.listdir(video_dir))):
        if not video_file.lower().endswith(".mp4"):
            continue

        video_id = os.path.splitext(video_file)[0]
        video_path = os.path.join(video_dir, video_file)
        output_video_path = os.path.join(output_video_dir, f"{video_id}.mp4")

        print(f"\n========== STARTED: {video_id} ==========")
        cap, total_frames, fps, w, h = setup_video_capture_1(video_path)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

        frame_count = 0
        pbar = tqdm(total=max_frames if max_frames else total_frames, desc=video_id)

        while True:
            ret, frame = cap.read()
            if not ret or (max_frames and frame_count >= max_frames):
                break

            annotated_bgr, boxes, class_ids = process_frame_1(frame, yolo_model, w, h, class_id_to_name)

            # ✅ Save original image
            img_filename = f'{video_id}_{frame_count:04d}.jpg'
            img_path = os.path.join(image_out_dir, img_filename)
            cv2.imwrite(img_path, frame)

            # ✅ Save YOLO-format label
            label_filename = f'{video_id}_{frame_count:04d}.txt'
            label_path = os.path.join(label_out_dir, label_filename)
            with open(label_path, 'w') as f:
                for box, cls_id in zip(boxes, class_ids):
                    x1, y1, x2, y2 = box
                    w_box = x2 - x1
                    h_box = y2 - y1
                    cx = x1 + w_box / 2
                    cy = y1 + h_box / 2
                    f.write(f"{int(cls_id)} {cx/w:.6f} {cy/h:.6f} {w_box/w:.6f} {h_box/h:.6f}\n")

            writer.write(annotated_bgr)
            frame_count += 1
            pbar.update(1)

        cap.release()
        writer.release()
        pbar.close()
        print(f"DONE: {video_id} — Processed {frame_count} frames")


In [11]:
def find_best_model(base_dir='runs_yolo/'):
    best_paths = list(Path(base_dir).rglob('best.pt'))
    if not best_paths:
        raise FileNotFoundError("No 'best.pt' file found in the 'runs/' directory.")
    
    # Optionally, sort by latest modified time
    best_paths.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    
    print(f"✅ Found best.pt at: {best_paths[0]}")
    return str(best_paths[0])


In [12]:
new_path = './runs/train/fine-tune-yolov8'
old_path = './runs/train/yolov8'

before_retrain = find_best_model(new_path)
after_retrain = find_best_model(old_path)

✅ Found best.pt at: runs\train\fine-tune-yolov8\weights\best.pt
✅ Found best.pt at: runs\train\yolov8\weights\best.pt


In [13]:
videos_predictions(before_retrain, class_id_to_name, video_dir='videos', output_base='./datasets/final_output_retrain', max_frames=1)

Model Used :: runs\train\fine-tune-yolov8\weights\best.pt


  0%|          | 0/11 [00:00<?, ?it/s]


========== STARTED: v1 ==========


v1:   0%|          | 0/1 [00:00<?, ?it/s]

DONE: v1 — Processed 1 frames

========== STARTED: v10 ==========


...

DONE: v6 — Processed 1 frames

========== STARTED: v8 ==========


v8:   0%|          | 0/1 [00:00<?, ?it/s]

DONE: v8 — Processed 1 frames

========== STARTED: v9 ==========


v9:   0%|          | 0/1 [00:00<?, ?it/s]

DONE: v9 — Processed 1 frames


In [14]:
videos_predictions(after_retrain, class_id_to_name, video_dir='videos', output_base='./datasets/final_output', max_frames=1)

Model Used :: runs\train\yolov8\weights\best.pt


  0%|          | 0/11 [00:00<?, ?it/s]


========== STARTED: v1 ==========


v1:   0%|          | 0/1 [00:00<?, ?it/s]

DONE: v1 — Processed 1 frames

========== STARTED: v10 ==========


...

DONE: v6 — Processed 1 frames

========== STARTED: v8 ==========


v8:   0%|          | 0/1 [00:00<?, ?it/s]

DONE: v8 — Processed 1 frames

========== STARTED: v9 ==========


v9:   0%|          | 0/1 [00:00<?, ?it/s]

DONE: v9 — Processed 1 frames
